Import Libraries and variables

In [123]:
import pandas as pd
import re

df=pd.read_csv("./datasets/cleaned_vehicles.csv")

noise_words=["brand new","b/n","brand n","brandnew","new","used","1st","1st owner","petrol","diesel","hybrid","electric","Anniversary",
"second","2nd owner","company maintained","first","owner","owners","manual",
"full option","full options","full-option","full-options","full optioned","full-option","highest spec","high spec","full spec","Pre Order","Moon Roof",
"moonroof","(brandnew)","(brand new)","(moonroof)"]


Add,remove and short

In [124]:
#body type
columns_to_add={
    "Body Type":"Model",
    "Seating Capacity":"Fuel Type",
    "Ground Clearance":"Seating Capacity",
    "Ground Clearance":"Ground Clearance (range)"
}

for col_name,ref_col in columns_to_add.items():
    if col_name not in df.columns:
        col_index=df.columns.get_loc(ref_col)+1
        df.insert(col_index,col_name,None)
    else:
        print(f"{col_name} column already exists")
#----------Modifing Dataset----------
#Lowercase Model
df["Model"]=df["Model"].str.lower().str.strip()
#Sort dataset
df=df.sort_values(by=["Manufacturer","Model","Year"],ascending=[True,True,False])
#----------Rearranging columns----------
#Year column after Model column
cols=list(df.columns)
cols.remove("Year")
cols.insert(cols.index("Model")+1,"Year")
df = df[cols]

#Fuel Type column after Year column
cols=list(df.columns)
cols.remove("Fuel Type")
cols.insert(cols.index("Year")+1,"Fuel Type")
df = df[cols]

#----------removing noise words from Model column----------
for word in noise_words:
    df["Model"]=df["Model"].str.replace(word,"",regex=True)
df["Model"]=df["Model"].str.replace(r"\s+"," ",regex=True).str.strip()

#----------removing duplicates----------
df=df.drop_duplicates(subset=["Manufacturer","Model","Year"],keep="first")

#----------Replace words in Model column----------
mask = (df["Manufacturer"].str.strip().str.lower() == "ford") & \
       (df["Model"].str.lower().str.contains("raptor ranger"))

df.loc[mask, "Model"] = df.loc[mask, "Model"].str.replace(
    r"raptor\s*ranger", "ranger raptor", regex=True, case=False
)

#----------Remove floating points----------
df["Seating Capacity"] = df["Seating Capacity"].astype("Int64")

Body Type column already exists
Seating Capacity column already exists
Ground Clearance column already exists


Adding features/Seating Capacity 

In [128]:
vehicle_map={
    "Audi":{
        "seating_capacity":{"a1":5,"a3":5,"a4":5,"a5":5,"a6":5,"a7":5,"a8":5,"q2":5,"q3":5,"q4":5,"q5":5,"q7":7,"q8":5,"e-tron":5},
        "body_type":{"sedan":"sedan","a1":"hatchback","a3":"hatchback","a3 sedan":"sedan","a3 tfsi sedan":"sedan","a3 sportsback":"hatchback",
                     "a4":"sedan/wagon","a5":"sedan","a6":"sedan/wagon","a7":"hatchback/hatchback sportsback","a8":"sedan","q2":"suv",
                     "q3":"suv/suv sportsback","q4":"suv/suv sportsback","q5":"suv/suv sportsback","q7":"suv","q8":"suv","e-tron":"suv/suv sportsback"},
        "ground_clearance":{"a1":"121-160","a3":"112-165","a4":"110-170","a5":"118-165","a6":"118-185","a7":"106-145"}
    },
    "BMW":{
        "seating_capacity":{"218i":5,"316i":5,"318i":5,"320d":5,"430i gran":5,"520d":5,"520i":5,"523i":5,"530e":5,"530i":5,"535i":5,
            "725d":5,"730d":5,"730ld":5,"730le":5,"740li":5,"f10":5,"active 7":5,"i3":4,"i5":5,"i7":5,"i8":4,
            "i8 roadster":2,"ix":5,"m760":5,"mini cooper":4,"x1":5,"x2":5,"x3":5,"x5":5,"x6":5,"x7":7,"z4":2},
        "body_type":{"218i":"coupe","316i":"sedan","318i":"sedan","320d":"sedan","430i gran":"coupe/gran coupe","520d":"sedan/touring",
            "520d gt":"hatchback","520i":"sedan/touring","523i":"sedan","530e":"sedan","530i":"sedan","535i":"sedan","528i":"sedan",
            "725d":"sedan","730ld":"sedan","740le":"sedan","740li":"sedan","740e":"sedan","750i":"sedan","active 7":"sedan",
            "i3":"hatchback","i5":"sedan","i7":"sedan","i8":"coupe/roadster","i8 roadster":"roadster","ix":"suv","m760":"sedan",
            "mini cooper":"hatchback","x1":"suv","x2":"suv","x3":"suv","x5":"suv","x6":"suv","x7":"suv","z4":"roadster"},
        "ground_clearance":{}
    },
    "BYD":{
        "seating_capacity":{"e6":5,"seal":5,"sealion":5,"atto":5},
        "body_type":{"e6":"sedan/hatchback","seal":"sedan","sealion":"suv","atto":"suv"},
        "ground_clearance":{}
    },
    "Chery":{
        "seating_capacity":{"qq":4,"qq 308":5,"tiggo":5},
        "body_type":{"qq":"hatchback","qq 308":"hatchback","tiggo":"suv"},
        "ground_clearance":{}
    },
    "Chevrolet":{
        "seating_capacity":{"cruze":5},
        "body_type":{"cruze":"hatchback"},
        "ground_clearance":{}
    },
    "DFSK":{
        "seating_capacity":{"580":7,"glory":7},
        "body_type":{"580":"suv","glory":"suv"},
        "ground_clearance":{}
    },
    "Daihatsu":{
        "seating_capacity":{"mira":4,"move":4,"rocky":5,"taft":5,"tanto":4,"terios":7,"thor":5},
        "body_type":{"mira":"hatchback","move":"hatchback","rocky":"suv","taft":"suv","tanto":"hatchback","terios":"suv","thor":"suv"},
        "ground_clearance":{}
    },
    "Datsun":{
        "seating_capacity":{"redi-go":5},
        "body_type":{"redi-go":"hatchback"},
        "ground_clearance":{}
    },
    "Dongfeng":{
        "seating_capacity":{"box":5},
        "body_type":{"box":"suv"},
        "ground_clearance":{}
    },
    "Fiat":{
        "seating_capacity":{"punto":5},
        "body_type":{"punto":"hatchback"},
        "ground_clearance":{}
    },
    "Ford":{
        "seating_capacity":{"everest":7,"fiesta":5,"kuga":5,"mustang 5liter":4,"mustang mach-e":5,"ranger":5,"wildtrak":5,"raptor":5,},
        "body_type":{"everest":"suv","fiesta":"hatchback","kuga":"suv","mustang 5liter":"coupe/convertible","mustang mach-e":"suv",
            "ranger":"pickup","wildtrak":"pickup","raptor":"pickup"},
        "ground_clearance":{}
    },
    "Honda":{
        "seating_capacity":{"civic":5,"cr-v":5,"city":5,"cr-v":5,"cr-v australian 7 seater":7,"cr-v vti l 7 seater":7,"cr-v vti lx 7 seater":7,
                 "hr-v":5,"jazz":5,"vezel":5,"cr-z zf1":2,"fit":5,"freed":7,"grace":5,"insight":5,"n-box":4,"n-one":4,"n-wgn":4,
                 "step wagon":7,"wr-v":5,"zrv":5},
        "body_type":{"civic":"sedan","cr-v":"suv","city":"sedan","cr-v 7 seater":"suv","cr-v australian 7 seater":"suv",
            "cr-v vti l 7 seater":"suv","cr-v vti lx 7 seater":"suv","hr-v":"suv","jazz":"hatchback","vezel":"suv","wr-v":"suv",
            "zr-v":"suv","cr-z zf1":"coupe","fit":"hatchback","fit shuttle":"wagon","fit shuttle gp2":"wagon","fit shuttle gp7":"wagon",
            "fit shuttle gp7 navi ":"wagon","fit shuttle gp2 navi":"wagon","freed":"minivan","grace":"sedan","insight":"sedan","n-box":"kei/hatchback",
            "n-one":"kei/hatchback","n-wgn":"kei/hatchback","step wagon":"minivan"},
        "ground_clearance":{}
    },
    "Hyundai":{
        "seating_capacity":{"accent":5,"creta":5,"eon":5,"elantra":5,"grand i10":5,"i20":5,"i30":5,"kona":5,"santa fe":7,"sonata":5,"tucson":5,
                            "ioniq":5,"terracan":5,"venue":5},
        "body_type":{"accent":"sedan/hatchback","creta":"suv","elantra":"sedan","eon":"hatchback","grand i10":"hatchback","i20":"hatchback",
                     "ioniq":"hatchback","i30":"hatchback/wagon","kona":"suv","santa fe":"suv","sonata":"sedan","tucson":"suv","terracan":"suv",
                     "venue":"suv"},
        "ground_clearance":{}
    },
    "Isuzu":{
        "seating_capacity":{"d-max":5,"mu-x":7},
        "body_type":{"d-max":"pickup","mu-x":"suv"},
        "ground_clearance":{}
    },
    "JAC":{
        "seating_capacity":{"t9":5},
        "body_type":{"t9":"pickup"},
        "ground_clearance":{}
    },
    "Jaecoo":{
        "seating_capacity":{"j7 4wd":5,"j6 ev":5,"j7 phev":5},
        "body_type":{"j7 4wd":"suv","j6 ev":"suv","j7 phev":"suv"},
        "ground_clearance":{}
    },
    "JMEV":{
        "seating_capacity":{"elight":5},
        "body_type":{"elight":"sedan/coupe"},
        "ground_clearance":{}
    },
    "Jaguar":{
        "seating_capacity":{"e-pace":5,"f-pace":5,"xf":5},
        "body_type":{"e-pace":"compact suv","f-pace":"suv","xf":"sedan"},
        "ground_clearance":{}
    },
    "Jeep":{
        "seating_capacity":{"compass":5,"grand cherokee":5,"renegade":5,"wrangler":4,"wrangler gladiator":5,"wrangler rubicon":5},
        "body_type":{"wrangler":"suv","grand cherokee":"suv","renegade":"suv","wrangler":"suv","wrangler gladiator":"pickup",
                     "wrangler rubicon":"suv"},
        "ground_clearance":{}
    },
    "Kia":{
        "seating_capacity":{"carens clavis":7,"carnival":7,"cerato":5,"ev5":5,"jeep caren":7,"picanto":5,"rio":5,"seltos":5,
                            "sonet":5,"sorento":7,"soul":5,"sportage":5,"stonic":5},
        "body_type":{"carens clavis":"muv","carnival":"muv","cerato":"sedan/hatchback","ev5":"uv","jeep caren":"muv",
                      "picanto":"hatchback","rio":"hatchback/sedan","seltos":"suv","sonet":"suv","sorento":"suv","soul":"hatchback",
                      "sportage":"suv","stonic":"suv"},
        "ground_clearance":{}
    },
    "Land Rover":{
        "seating_capacity":{"defender":5,"defender 110":5,"defender 130":7,"discovery 4":7,
                            "discovery 5":7,"discovery sport":5,"freelander":5,"range":5,"range lwb":7,},
        "body_type":{"defender":"suv","defender 110":"suv","defender 130":"suv","discovery 4":"suv",
                     "discovery 5":"suv","discovery sport":"suv","freelander":"suv","range":"suv"},
        "ground_clearance":{}
    },
    "Lexus":{
        "seating_capacity":{"lbx":5,"lbx elegant":5,"lm500h direct 4":7,"ls500 executive":5,"ls600h":5,"ls600h lwb":5,
                            "lx500d":5,"lx570":5,"lx600":7,"lx600 fully loaded":7,"lx600 maximum loaded":7,"rx350":5,"rx400":5,
                            "rx400h":5,"rx450h":5},
        "body_type":{
            "lbx":"suv","lbx elegant":"suv","lm500h direct 4":"muv","ls500 executive":"sedan","ls600h":"sedan",
            "ls600h lwb":"sedan","lx500d":"suv","lx570":"suv","lx600":"suv","lx600 fully loaded":"suv",
            "lx600 maximum loaded":"suv","rx350":"suv","rx400":"suv","rx400h":"suv","rx450h":"suv"

        },
        "ground_clearance":{}
    },
    "MG":{
        "seating_capacity":{"ep plus":5,"mg4 ":5,"zs":5},
        "body_type":{"ep plus":"station wagon","mg4":"hatchback","zs ev":"suv","zs":"suv",},
        "ground_clearance":{}
    },
    "Mahindra":{
        "seating_capacity":{"bmt plus vx":2,"bolero":2,"e2o":4,"genio double cab":5,"kuv100":6,"scorpio":7,"xuv 3xo":5,"xuv500":7},
        "body_type":{"bmt plus vx":"pickup","bolero":"pickup","e2o":"hatchback","genio double cab":"pickup","kuv100":"suv",
                     "scorpio":"suv","xuv 3xo":"suv","xuv500":"suv"},
        "ground_clearance":{}
    },
    "Maruti Suzuki":{
        "seating_capacity":{"800":4,"alto":5,"ritz":5,"wagon r":5,"zen":5},
        "body_type":{"800":"hatchback","alto":"hatchback","ritz":"hatchback","wagon r":"hatchback","zen":"hatchback",}

    },
    "Mazda":{
        "seating_capacity":{"2":5,"3":5,"3 bm":5,"6":5,"6 skyactiv":5,"axela":5,"axela skyactiv":5,"axela skyactiv-sl":5,
                            "cx-5":5,"demio":5,"familia":5,"flair":5},
        "body_type":{"2":"hatchback/sedan","3":"hatchback/sedan","3 bm":"hatchback/sedan","6":"sedan/wagon","6 skyactiv":"sedan/wagon",
                     "axela":"sedan/hatchback","axela skyactiv":"sedan/hatchback","axela skyactiv-sl":"sedan/hatchback","familia":"sedan/wagon",
                     "demio":"hatchback/sedan","cx-5":"suv","flair":"kei/hatchback"},
        "ground_clearance":{}
    },
    "Mercedes Benz":{
        "seating_capacity":{"a180":5,"250e":5,"a45":5,"b250e":5,"c160":5,"c180":5,"c200":5,"c220":5,"c230":5,"c350":5,"cla":5,"cla45":5,
                            "e200":5,"e200 cabriolet":4,"g450d":5,"g400d":5,"g800":5,"gla":5,"glb":7,"gle 300d 7 seater":7,"gle 350d 7 seater":7,
                            "gle 300d":5,"gle 300d airmatic premium plus":5,"gle 400 amg premium plus":5,"gls":7,"ml350":5,"s300":5,"s350":5,"s400":5,
                            "s500":5,"s580":5,"vito":9,"vito 116 cdi mixto":6,"x250d":5},
        "body_type":{"a180":"hatchback","250e":"hatchback/sedan","a45":"hatchback","b250e":"hatchback","c160":"sedan","c180":"sedan","c200":"sedan",
                     "c220":"sedan","c230":"sedan","c350":"sedan","cla":"sedan","e200":"sedan","e200 cabriolet":"convertible","g450d":"suv","vito":"van",
                     "g400d":"suv","g800":"suv","gla":"suv","glb":"suv","gle":"suv","gls":"suv","ml350":"suv","s300":"sedan","s350":"sedan","s400":"sedan",
                     "s500":"sedan","s580":"sedan","x250d":"pickup"},
        "ground_clearance":{}
    },
    "Micro":{
        "seating_capacity":{"panda":4,"actyon 2000":5,"actyon cab":5,"actyon sports":5,"almaz":7,"baic x25":5,"chery tiggo pro4":5,"korondo":5,"kyron":7,
                            "mx 7":5,"rexton":7,"tivoli":5,"trend":5},
        "body_type":{"panda":"hatchback","actyon 2000":"suv","actyon cab":"pickup","actyon sports":"suv","almaz":"suv","baic x25":"suv","chery tiggo pro4":"suv",
                     "korondo":"suv","kyron":"suv","mx 7":"sedan","rexton":"suv","tivoli":"suv","trend":"hatchback"},
        "ground_clearance":{}
    },
    "Mitsubishi":{
        "seating_capacity":{"attrage":5,"delica mini":4,"eclipse cross":5,"ek custom":4,"ek space":4,"ek wagon":4,"ek x":5,"l200":5},
        "body_type":{"attrage":"sedan","delica mini":"kei/minivan","eclipse cross":"suv","ek custom":"hatchback","ek space":"hatchback","ek wagon":"hatchback",
                     "ek x":"suv","l200":"pickup"},
        "ground_clearance":{}
    },
    

}


In [129]:


for brand, attributes in vehicle_map.items():
    for model, seating_capacity in attributes["seating_capacity"].items():
        df.loc[
            (df["Manufacturer"].str.strip().str.lower() == brand.strip().lower())
            & (df["Model"].str.contains(model, case=False, regex=False)),
            "Seating Capacity",
        ] = seating_capacity

    for model, btype in attributes["body_type"].items():
        mask = (df["Manufacturer"] == brand) & (df["Model"].str.lower().eq(model))
        df.loc[mask, "Body Type"] = btype
        mask = (df["Manufacturer"] == brand) & (
            df["Model"].str.lower().str.contains(model)
        )
        df.loc[mask & df["Body Type"].isna(), "Body Type"] = btype
    

    print(f"Total {brand} cars: {df[df['Manufacturer'] == brand].shape[0]}")

Total Audi cars: 86
Total BMW cars: 133
Total BYD cars: 6
Total Chery cars: 6
Total Chevrolet cars: 1
Total DFSK cars: 18
Total Daihatsu cars: 105
Total Datsun cars: 2
Total Dongfeng cars: 1
Total Fiat cars: 1
Total Ford cars: 55
Total Honda cars: 211
Total Hyundai cars: 40
Total Isuzu cars: 1
Total JAC cars: 2
Total Jaecoo cars: 3
Total JMEV cars: 1
Total Jaguar cars: 5
Total Jeep cars: 9
Total Kia cars: 70
Total Land Rover cars: 112
Total Lexus cars: 16
Total MG cars: 8
Total Mahindra cars: 13
Total Maruti Suzuki cars: 15
Total Mazda cars: 23
Total Mercedes Benz cars: 136
Total Micro cars: 39
Total Mitsubishi cars: 124


In [130]:
df.to_csv("./datasets/cleaned_vehicles.csv",index=False)
print(f"Saved {len(df)} records to cleaned_vehicles.csv")

Saved 3205 records to cleaned_vehicles.csv
